<a href="https://colab.research.google.com/github/ab1470/Vehicle-Detection/blob/master/Vehicle_Detector_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Train a Core ML Object Detection Model to Recognize Automobiles**

This notebook uses Apple's [Turi Create](https://github.com/apple/turicreate) to train an object detection model that recognizes automobiles. The training data are images from the [Stanford Cars Dataset](https://ai.stanford.edu/~jkrause/cars/car_dataset.html); each image is annotated with the bounding box coordinates for any automobiles within the image.

The end resusult is a `.mlmodel` file that, when given an image input, will output normalized bounding box coordinates for any predicted automobiles within the image, as well as the confidence value for each prediction. The `.mlmodel` file can then be used in any app that supports Apple's [Core ML framework](https://developer.apple.com/documentation/coreml).

I created and ran this notebook in [Google Colaboratory](https://colab.research.google.com/notebooks/intro.ipynb), which is a free cloud-based Jupyter notebook environment, and includes access to a GPU for training ML models.

### **1) Configure your environment**

##### Install the necessary packages

The model was trained using Turi Create version 5.8. At time of writing, I was unable to successfully use version 6.x because it would crash the Colab notebook (similar to what was described [here](https://github.com/apple/turicreate/issues/2917))

In [0]:
!pip install turicreate==5.8

# The wrong version of MXNET will be installed; uninstall it
!pip uninstall -y mxnet

# Install CUDA10-compatible version of mxnet 
!pip install mxnet-cu100==1.5.1

**BEFORE YOU CONTINUE: Restart the runtime**

The version of the numpy package that Turi Create installed is different from the version supplied by default in the Colab notebook, so you must restart the runtime in order to use the new version. Either click the "RESTART RUNTIME" button in the output above, or click Runtime -> Restart runtime... in the menu bar.

In [0]:
import turicreate as tc
import os

# tell Turi Create to use all available GPUs
tc.config.set_num_gpus(-1)

In [0]:
# create helper methods to delete files and directories
import shutil

def delete_dir(directory: str):
  if os.path.isdir(directory):
    shutil.rmtree(directory)
  else:
    print(f"'{directory}' is not a directory")

def delete_file(filepath: str):
  if os.path.isfile(filepath):
    !rm $filepath
  else:
    print(f"'{filepath}' is not a valid file path")

# delete the "sample_data" directory that's included in the Colab notebook
delete_dir("sample_data")

##### Mount Google Drive and create a directory for the files

In [0]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# Create a directory to store all files for this project
# (change the `parent_dir` and `directory` variables if you want to use a different location)
parent_dir = "/content/gdrive/My Drive"
directory = "Vehicle Detection Model"
path = os.path.join(parent_dir, directory) 

if not os.path.isdir(path):
  os.mkdir(path)

# Set the current working directory to the one you just made
os.chdir(path)

### **2) Download data**

Download the compressed images and their annotations from the [Stanford Cars Dataset](https://ai.stanford.edu/~jkrause/cars/car_dataset.html), then unzip the images

In [0]:
!wget "http://imagenet.stanford.edu/internal/car196/car_ims.tgz"
!wget "http://imagenet.stanford.edu/internal/car196/cars_annos.mat"


import tarfile

# extract the images from the tarfile
tarred_file = tarfile.open("car_ims.tgz")
tarred_file.extractall()
tarred_file.close()

# delete the tarfile
delete_file("car_ims.tgz")

### **3) Save the data into an SFrame**

We'll store our images and annotations in an [SFrame](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.html), which Turi Create will use when it trains the model. 

##### First, we need to convert the annotations into a format that Turi Create can parse:

In [0]:
# install the scipy package
!pip install -U -q scipy

# load the annotations from the .mat file
from scipy import io
annos_mat = io.loadmat('cars_annos.mat')
annotations = annos_mat['annotations'][0]

# helper function to convert the annotations to a Turi Create-compatible format
def convert_annotation(_row):
    label = _row[0][0];
    x_min = int(_row[1][0][0]) # left x
    y_min = int(_row[2][0][0]) # top y
    x_max = int(_row[3][0][0]) # right x
    y_max = int(_row[4][0][0]) # bottom y

    result = {'coordinates':
        {
            'x' : (x_min + x_max)/2,
            'y' : (y_min + y_max)/2,
            'width': x_max - x_min,
            'height': y_max - y_min
        },
            'label': "car"
        }
    return result

# convert all of the annotations and store them in a hash table
annotations_map = {}

for anno in annotations:
  image_name = anno[0][0].split("/")[-1].split(".jpg")[0]
  converted_anno = convert_annotation(anno)
  annotations_map[image_name] = converted_anno

In [0]:
# Print the annotations for images '000001' and '000002' to verify the conversion:
print(annotations_map['000001'])
print(annotations_map['000002'])

{'coordinates': {'x': 482.5, 'y': 362.0, 'width': 741, 'height': 710}, 'label': 'car'}
{'coordinates': {'x': 244.5, 'y': 113.0, 'width': 393, 'height': 178}, 'label': 'car'}


##### Now load the images and their annotations into an SFrame:

In [0]:
# load images into a new SFrame
image_sframe = tc.image_analysis.load_images("car_ims", recursive=True, random_order=True)

# create an 'image_name' column in the SFrame to hold the image's filename (without the '.jpg' extension)
image_sframe['image_name'] = image_sframe['path'].apply(lambda x: x.split('/')[-1].split('.jpg')[0])

# create an 'annotations' column in the SFrame to hold the converted annotation values, 
# which we'll take from the annotations_map dict created above
image_sframe['annotations'] = image_sframe['image_name'].apply(lambda x: annotations_map[x])

##### Examine the SFrame to ensure that everything looks good:

In [0]:
# first, look at the first 10 entries in the SFrame
image_sframe.head()

In [0]:
# Now add a column to the SFrame to hold the image with the ground truth bounding box drawn on top
image_sframe['image_with_ground_truth'] = tc.object_detector.util.draw_bounding_boxes(image_sframe['image'], image_sframe['annotations'])

In [0]:
# Note: if you were running this notebook on your local machine, you could call 
# `image_sframe.explore()`, which would launch a new window that visualizes the 
# SFrame (including the images). You would also be able to do this within the 
# Colab notebook if it were running Turi Create version 6+. If neither if these 
# is the case, though, you will need to view the images one-by-one.

# Display the ground truth bounding boxes for the first 10 images
from PIL import Image
for i in range(3):
  tc_img = image_sframe['image_with_ground_truth'][i]
  img = Image.fromarray(tc_img.pixel_data)
  display(img)

In [0]:
# If everything looks good, remove the `image_with_ground_truth` column from the SFrame
image_sframe = image_sframe.remove_column('image_with_ground_truth')

In [0]:
# Because the images and annotations are now loaded into an SFrame, we can delete
# them from the current directory
delete_dir("car_ims")
delete_file("cars_annos.mat")

##### Now split the SFrame into two SFrames, one for the training data and one for the test data, then save the two SFrames:

In [0]:
# Split the data
train_data, test_data = image_sframe.random_split(0.8)

In [0]:
# Save the training and test SFrames so you can use them later
train_data.save('train_data.sframe')
test_data.save('test_data.sframe')

###**4) Train the model**

##### Optional:

If you have restarted the notebook and you need to re-load the SFrames that you created in steps 2 & 3, you can do so with the following code:

In [0]:
# Make sure the current working directory is the one where you stored the SFrames
train_data =  tc.SFrame('train_data.sframe')
test_data =  tc.SFrame('test_data.sframe')

##### **Start the training**

The training will take a long time! For me, it ran for about 9 hours.

Note: if the `max_iterations` parmameter is not set, the `tc.object_detector.create` method will automatically determine the number of iterations based on the amount of data provided. 

If you want to limit the number of iterations (for example, if you want to do a test run before starting the full training), set the max_iterations parameter like so:


```
model = tc.object_detector.create(train_data, max_iterations = 200)
```



In [0]:
# Train the model
model = tc.object_detector.create(train_data)
# model = tc.object_detector.create(train_data)

# Create names for .model and .mlmodel files
model_name = "VehicleDetector.model"
mlmodel_name = "VehicleDetector.mlmodel"

# Save the model for later use in Turi Create
model.save(model_name)

# Export and save the CoreML model
model.export_coreml(mlmodel_name)

###**5) Evaluate the model**

##### Optional:

If you need to reload the model and SFrame files, use the following code:

In [0]:
# Make sure the current working directory is correct
model = tc.load_model("VehicleDetector.model")
test_data =  tc.SFrame('test_data.sframe')

##### Evaluate the model on the test data, then print the metrics:

In [0]:
# Evaluate the model and save the results into a dictionary
# (it may take ~10 minutes)
metrics = model.evaluate(test_data)
print(metrics)

##### Run predictions on the test data, then add a column to the SFrame with the bounding boxes for the predictions drawn on top of the images:

In [0]:
# Save predictions to an SArray
# (it may take ~10 minutes)
test_data['predictions'] = model.predict(test_data)
test_data['image_with_predictions'] = tc.object_detector.util.draw_bounding_boxes(test_data['image'], test_data['predictions'])

# Save the updated test_data SFrame
test_data.save('test_data.sframe')

In [0]:
# Display predictions for the first 10 images
from PIL import Image
for i in range(10):
  tc_img = test_data['image_with_predictions'][i]
  img = Image.fromarray(tc_img.pixel_data)
  display(img)

###**6) Reduce the model's size** (optional)

You can reduce the size of the `.mlmodel` file by using a lower precision for the weight parameters. See '[Reducing the Size of your Core ML App](https://developer.apple.com/documentation/coreml/reducing_the_size_of_your_core_ml_app)' for an explanation. 

In [0]:
import coremltools

model_spec = coremltools.utils.load_spec('VehicleDetector.mlmodel')
model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
coremltools.utils.save_spec(model_fp16_spec, 'VehicleDetectorSM.mlmodel')

In [0]:
from coremltools.models.neural_network import quantization_utils

model = coremltools.models.MLModel('VehicleDetector.mlmodel')
quantized_model = quantization_utils.quantize_weights(model, 8, 'linear')
# quantized_model = coremltools.models.MLModel(quantized_model)
coremltools.utils.save_spec(quantized_model, 'VehicleDetectorSM8.mlmodel')

###**7) Download files to your computer** (optional)

To download an individual file, you can use the code below, or you can right-click the file in the file tree to the left

To download a directory, you must use the code below to compress the file. Then you can download the compressed file either with code or with the file tree.

In [0]:
# import the files package
from google.colab import files

In [0]:
# Download the .mlmodel file to your computer
files.download('VehicleDetector.mlmodel') 

In [0]:
files.download('VehicleDetectorSM.mlmodel')
files.download('VehicleDetectorSM8.mlmodel')

In [0]:
# Compress the `test_data` SFrame and download it to your computer
!zip -r test_data.zip test_data.sframe
files.download('test_data.zip')

###**8) Housekeeping**

The files created in this process take up a lot of space on your Google Drive! You should delete the files that you don't need to keep.

In [0]:
# Delete the SFrames and the Turi Create .model file
delete_dir("train_data.sframe")
delete_dir("test_data.sframe")
delete_dir("VehicleDetector.model")

In [0]:
# If you don't need to keep *any* of the files on your Google Drive, you can delete the entire directory
# Make sure you've downloaded whichever files you want to keep to your computer
delete_dir(path)